# Calculate hypsometric indexes

Hypsometric index (HI) defined by Jiskoot et al. (2000): 

$ HI = ({H_{max} - H_{med}}) / (H_{med} - H_{min}) $

And if $0 < HI < 1$, $HI = -1 / HI$

where $H_{max}$ and $H_{min}$ are the maximum and minimum glacier elevations and $H_{med}$ the elevation of the contour line that divides the glacier area in half.

| Value | Category |
| -------- | ------- |
| HI < -1.5  | Very top-heavy |
| -1.5 < HI < 1.2 | Top-heavy |
| -1.2 < HI < 1.2 | Equidimensional |
| 1.2 < HI < 1.5 | Bottom-heavy |
| HI > 1.5 | Very bottom-heavy |

In [ ]:
import os
import glob
import geopandas as gpd
import pandas as pd
import xarray as xr
import rioxarray as rxr
from tqdm.auto import tqdm
import numpy as np

In [ ]:
# Define paths in directory
study_sites_path = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/study-sites/'
out_path = os.path.join(study_sites_path, '..', 'analysis')

# Grab site names
rgi_ids = [x for x in sorted(os.listdir(study_sites_path)) if 'RGI' in x]
print(f'{len(rgi_ids)} sites:')
rgi_ids

In [ ]:
# Define function for calculating HI
def calculate_hypsometric_index(dem):
    # Calculate HI
    hmin = float(dem.min().data)
    hmax = float(dem.max().data)
    hmed = float(dem.median().data)
    
    hi = (hmax - hmed) / (hmed - hmin)
    if (hi < 1) & (hi > 0):
        hi = -1/hi

    # Determine HI category
    if hi < -1.5:
        hi_category = 'very top-heavy'
    elif (hi > -1.5) & (hi < -1.2):
        hi_category = 'top-heavy'
    elif (hi > -1.2) & (hi < 1.2):
        hi_category = 'equidimensional'
    elif (hi > 1.2) & (hi < 1.5):
        hi_category = 'bottom-heavy'
    elif hi > 1.5:
        hi_category = 'very bottom-heavy'
    else:
        hi_category = 'None'
    
    return hi, hi_category

# Define output file name
his_fn = os.path.join(out_path, 'hypsometric_indexes.csv')

# Check if already exists in directory
if os.path.exists(his_fn):
    print('Hypsometric indexes already exist in file, skipping.')
else:
    # Initialize dataframe of HIs
    his_df = pd.DataFrame()
    
    # Iterate over sites
    for rgi_id in tqdm(rgi_ids):
        # Load glacier boundaries
        aoi_fn = os.path.join(study_sites_path, rgi_id, 'AOIs', f'{rgi_id}_outline.shp')
        aoi = gpd.read_file(aoi_fn)
        aoi = aoi.to_crs('EPSG:4326')
        
        # Load DEM
        dem_fn = glob.glob(os.path.join(study_sites_path, rgi_id, 'DEMs', '*.tif'))[0]
        dem = rxr.open_rasterio(dem_fn).isel(band=0)
        dem = dem.rio.reproject('EPSG:4326')
        dem = xr.where(dem < -1e10, np.nan, dem)
        dem = dem.rio.write_crs('EPSG:4326')
        
        # Clip DEM to AOI
        dem_clip = dem.rio.clip(aoi.geometry)
    
        # Calculate HI
        hi, hi_category = calculate_hypsometric_index(dem_clip)
    
        # Add to dataframe
        hi_df = pd.DataFrame({'RGIId': [rgi_id],
                              'HI': [hi],
                              'HI_category': [hi_category]})
        his_df = pd.concat([his_df, hi_df], axis=0)
    
    his_df.reset_index(drop=True, inplace=True)
    
    # Save to file
    his_df.to_csv(his_fn, index=False)
    print('Hypsometric indexes saved to file:', his_fn)

In [ ]:
import matplotlib.pyplot as plt

his_df = pd.read_csv(his_fn)
plt.hist(his_df['HI'], bins=100)
plt.show()

In [ ]:
rgi_id = his_df.loc[his_df['HI'] > 30, 'RGIId'].values[0]

# Load glacier boundaries
aoi_fn = os.path.join(study_sites_path, rgi_id, 'AOIs', f'{rgi_id}_outline.shp')
aoi = gpd.read_file(aoi_fn)
aoi = aoi.to_crs('EPSG:4326')

# Load DEM
dem_fn = glob.glob(os.path.join(study_sites_path, rgi_id, 'DEMs', '*.tif'))[0]
dem = rxr.open_rasterio(dem_fn).isel(band=0)
dem = dem.rio.reproject('EPSG:4326')
dem = xr.where(dem < -1e10, np.nan, dem)
dem = dem.rio.write_crs('EPSG:4326')

# Clip DEM to AOI
# dem_clip = dem.rio.clip(aoi.geometry)

fig, ax = plt.subplots()
dem.plot(ax=ax, cmap='terrain')
aoi.plot(ax=ax, facecolor='None', edgecolor='k')
plt.show()

In [ ]:
dem_fn